In [9]:
with open('data/train.txt') as f:
    train = f.readlines()
with open('data/dev.txt') as f:
    dev = f.readlines()

In [10]:
from tqdm import tqdm
train[-1] = train[-1] + '\n'
train_dev = train + dev
with open('data/train_dev.txt', 'w') as f:
    for data in tqdm(train_dev):
        f.write(data)
len(train_dev)
    

100%|██████████| 11000/11000 [00:00<00:00, 687334.73it/s]


11000

In [11]:
with open('data/class.txt') as f:
    classes = f.readlines()
label2class = {}
for class_ in classes:
    class_ = class_.replace('\n', '').split('   ')
    label2class[int(class_[1])] = class_[0]
class2label = {}
for key, value in label2class.items():
    class2label[value] = key
label2class

{0: 'Diagnosis',
 1: 'Treatment',
 2: 'Common Knowledge',
 3: 'healthy lifestyle',
 4: 'Epidemiololgy',
 5: 'Other'}

In [12]:
target_json = []
for data in train_dev:
    tmp = {}
    conversations = []
    data = data.replace('\t', '').replace('\n', '')
    text = data[:-1]
    label = int(data[-1])
    conversations.append({'from': 'human', 'value': text})
    conversations.append({'from': 'gpt', 'value': label2class[label]})
    tmp['conversations'] = conversations
    target_json.append(tmp)

In [13]:
len(target_json), target_json[0]

(11000,
 {'conversations': [{'from': 'human', 'value': '空腹血糖78，是否属于糖尿病范围?'},
   {'from': 'gpt', 'value': 'Diagnosis'}]})

In [16]:
import json

with open('data/train_dev_11k.json', 'w', encoding='utf-8') as f:
    json.dump(target_json, f)

In [17]:
json.load(open('data/train_dev_11k.json'))

[{'conversations': [{'from': 'human', 'value': '空腹血糖78，是否属于糖尿病范围?'},
   {'from': 'gpt', 'value': 'Diagnosis'}]},
 {'conversations': [{'from': 'human', 'value': '减肥后是否能改善糖尿病状况?'},
   {'from': 'gpt', 'value': 'healthy lifestyle'}]},
 {'conversations': [{'from': 'human', 'value': '患有糖尿病的母亲而父亲没有，是否会遗传给下一代?'},
   {'from': 'gpt', 'value': 'Common Knowledge'}]},
 {'conversations': [{'from': 'human', 'value': '糖尿病是否会引起眼睛水肿?'},
   {'from': 'gpt', 'value': 'Epidemiololgy'}]},
 {'conversations': [{'from': 'human', 'value': '妊娠期糖尿病的注意事项是什么?'},
   {'from': 'gpt', 'value': 'Common Knowledge'}]},
 {'conversations': [{'from': 'human', 'value': '糖尿病患者是否可以用党参泡水喝?'},
   {'from': 'gpt', 'value': 'healthy lifestyle'}]},
 {'conversations': [{'from': 'human', 'value': '患有胆结石且有糖尿病，应该如何处理?'},
   {'from': 'gpt', 'value': 'Treatment'}]},
 {'conversations': [{'from': 'human', 'value': '糖尿病心血管并发症应该如何处理?'},
   {'from': 'gpt', 'value': 'Treatment'}]},
 {'conversations': [{'from': 'human', 'value': '糖尿病引起骨髓炎应该如何处理?'}

In [8]:
classes

['Diagnosis   0\n',
 'Treatment   1\n',
 'Common Knowledge    2\n',
 'healthy lifestyle   3\n',
 'Epidemiololgy   4\n',
 'Other   5']

In [30]:
from transformers import AutoTokenizer, BertForSequenceClassification
from torch import nn
import torch
tokenizer = AutoTokenizer.from_pretrained("ckpts/bert-base-chinese")
model = BertForSequenceClassification.from_pretrained("ckpts/bert-base-chinese")
model.classifier = nn.Linear(768, 6)
model.config.num_labels = 6
model.num_labels = 6

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckpts/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
import torch
text = '糖尿病患者是否可以用党参泡水喝?'
inputs = tokenizer(text, return_tensors='pt', padding=True)
inputs['labels'] = torch.LongTensor((3,)).reshape(1, 1)

In [34]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [35]:
out = model(**inputs)

In [36]:
out.keys()

odict_keys(['loss', 'logits'])

In [37]:
out.loss

tensor(1.5866, grad_fn=<NllLossBackward0>)